In [1]:
!pip install transformers datasets peft accelerate bitsandbytes trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 24.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 99.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 

In [ ]:
from huggingface_hub import login
login(token="----")

In [22]:
# ✅ 1. Set up environment
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import load_dataset
import torch

# Ensure GPU is being used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [23]:
# ✅ 2. Load tokenizer and model
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",  # automatically maps model to GPU if available
    trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
# ✅ 3. Apply LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)
model = get_peft_model(model, lora_config)

In [25]:
# ✅ 4. Load dataset
dataset = load_dataset("json", data_files="/kaggle/input/jsonfiles/ISO_Format_Dataset.jsonl")  # <-- Change this to your JSON file
train_data = dataset["train"]

Generating train split: 0 examples [00:00, ? examples/s]

In [26]:
# ✅ 5. Format prompt
def format_instruction(example):
    instruction = str(example.get("instruction", ""))
    input_text = str(example.get("input", ""))
    output_text = str(example.get("output", ""))

    prompt = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output_text}
"""
    return {"text": prompt}

train_data = train_data.map(format_instruction)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [27]:
# ✅ 6. Tokenization
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_dataset = train_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [33]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# ✅ Fast TrainingArguments for quick test
training_args = TrainingArguments(
    output_dir="./mistral_datetime_lora_test",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    num_train_epochs=1,  # Only 1 epoch
    max_steps=5,  # ✅ Limit to 5 steps only
    logging_steps=1,
    save_strategy="no",  # ✅ Don't save checkpoints
    bf16=True if torch.cuda.is_available() else False,
    report_to="none",  # ✅ No external logging
    disable_tqdm=False,  # ✅ Show progress bar
    logging_dir="./logs_test",
)

# ✅ Dummy collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# ✅ Optional: Reduce dataset size for fast debug
small_train_dataset = tokenized_dataset.select(range(10))  # just 10 samples

# ✅ Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# ✅ Start fast training
trainer.train()


/tmp/ipykernel_36/3340328774.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,1.823300
2,1.624200
3,1.715800
4,1.635700
5,1.149500


TrainOutput(global_step=5, training_loss=1.5896893978118896, metrics={'train_runtime': 69.4964, 'train_samples_per_second': 0.072, 'train_steps_per_second': 0.072, 'total_flos': 109863984168960.0, 'train_loss': 1.5896893978118896, 'epoch': 0.5})

In [34]:
# Save the trained model
trainer.save_model()
print("Model saved to ./time-date-extractor/")

Model saved to ./time-date-extractor/


In [35]:
# Test inputs
test_inputs = [
    "tomorrow morning at 9:15",
    "next Friday between 2-4 pm",
    "this afternoon around 3:30",
    "last Tuesday at 11 am"
]

# Set model to evaluation mode
model.eval()

instruction = "Extract exact datetime from this expression assuming today is 2025-08-04."

print("Testing the fine-tuned model:")
print("=" * 60)

for test_input in test_inputs:
    # Format prompt for inference
    prompt = f"""### Instruction:
{instruction}

### Input:
{test_input}

### Output:
"""
    
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    
    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract only the generated part
    generated_text = response[len(prompt):].strip()
    
    print(f"Input: {test_input}")
    print(f"Output: {generated_text}")
    print("-" * 50)


Testing the fine-tuned model:


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2505: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Input: tomorrow morning at 9:15
Output: {'datetime': datetime.datetime(2025, 8, 5, 9, 15)}
--------------------------------------------------
Input: next Friday between 2-4 pm
Output: {'start': datetime.datetime(2025, 8, 9, 2, 0), 'end': datetime.datetime(2025, 8, 9, 4, 0)}

### Explanation:
Assuming today is 2025-08-04, next Friday would be 2025-08-09. The time range given is between 2-4 pm,
--------------------------------------------------
Input: this afternoon around 3:30
Output: {'start': datetime.datetime(2025, 8, 4, 15, 30), 'end': None}

### Explanation:
The datetime extracted from the given input is 3:30 PM on 2025-08-04. Since there is no information about an end time, the end time is set to None.
--------------------------------------------------
Input: last Tuesday at 11 am
Output: {'start': datetime.datetime(2025, 8, 6, 11, 0), 'end': None}

### Explanation:
Assuming last Tuesday means the Tuesday before today, which is 2025-08-06. The time given is 11 am, so the start tim

In [36]:
# If you want to test prompt formatting without training, run this cell instead
test_cases = [
    "yesterday evening around 8:30",
    "this morning at 7-10", 
    "show it on 26th April at 10"
]

instruction = "Extract exact datetime from this expression assuming today is 2025-08-04."

print("Testing prompt formatting:")
for case in test_cases:
    prompt = f"""### Instruction:
{instruction}

### Input:
{case}

### Output:
"""
    print(f"Input: {case}")
    print("Formatted prompt:")
    print(prompt)
    print("-" * 50)


Testing prompt formatting:
Input: yesterday evening around 8:30
Formatted prompt:
### Instruction:
Extract exact datetime from this expression assuming today is 2025-08-04.

### Input:
yesterday evening around 8:30

### Output:

--------------------------------------------------
Input: this morning at 7-10
Formatted prompt:
### Instruction:
Extract exact datetime from this expression assuming today is 2025-08-04.

### Input:
this morning at 7-10

### Output:

--------------------------------------------------
Input: show it on 26th April at 10
Formatted prompt:
### Instruction:
Extract exact datetime from this expression assuming today is 2025-08-04.

### Input:
show it on 26th April at 10

### Output:

--------------------------------------------------
